In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import os
from sklearn.model_selection import train_test_split
import pandas as pd
from pydataset import data
from env import get_db_url
import pandas as pd
import numpy as np



## Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [2]:
def new_zillow_data():
    """
    Fetches Zillow data from the database using SQL query.

    Returns:
    - df: DataFrame containing Zillow data
    """
   
    conn = get_db_url('zillow')

    query = '''
        SELECT *
    FROM properties_2017 AS p
    JOIN predictions_2017 AS pr ON p.parcelid = pr.parcelid
    LEFT JOIN airconditioningtype AS act ON p.airconditioningtypeid = act.airconditioningtypeid
    LEFT JOIN architecturalstyletype AS ast ON p.architecturalstyletypeid = ast.architecturalstyletypeid
    LEFT JOIN buildingclasstype AS bct ON p.buildingclasstypeid = bct.buildingclasstypeid
    LEFT JOIN heatingorsystemtype AS hot ON p.heatingorsystemtypeid = hot.heatingorsystemtypeid
    LEFT JOIN propertylandusetype AS plt ON p.propertylandusetypeid = plt.propertylandusetypeid
    LEFT JOIN storytype AS st ON p.storytypeid = st.storytypeid
    LEFT JOIN typeconstructiontype AS tct ON p.typeconstructiontypeid = tct.typeconstructiontypeid
    LEFT JOIN unique_properties AS up ON p.parcelid = up.parcelid;
    
    '''

    df = pd.read_sql(query, conn)
    return df

## Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [3]:
def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
    """
    Drops columns and rows from a DataFrame based on the percentage of missing values.
    
    Args:
        df (pandas.DataFrame): The DataFrame to handle missing values.
        prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
        prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).
        
    Returns:
        pandas.DataFrame: The DataFrame with dropped rows and columns.
    """
    
    threshold_col = int(round(prop_required_column * len(df.index)))
    df = df.dropna(thresh=threshold_col, axis=1)
    
  
    threshold_row = int(round(prop_required_row * len(df.columns)))
    df = df.dropna(thresh=threshold_row)
    
    return df


In [4]:
def missing_value_summary(df):
    """
    Returns a DataFrame summarizing the count and percentage of missing values for each attribute in the input DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame of observations and attributes.

    Returns:
        pandas.DataFrame: The summary DataFrame with count and percentage of missing values for each attribute.
    """
    missing_counts = df.isnull().sum()
    missing_percentages = (missing_counts / len(df)) * 100
    summary_df = pd.DataFrame({
        'Attribute': missing_counts.index,
        'Missing Count': missing_counts.values,
        'Missing Percentage': missing_percentages.values
    })
    return summary_df




## The input:

## A dataframe

In [5]:
df = new_zillow_data()

In [6]:
df.fips.value_counts()

6037.0    50697
6059.0    20631
6111.0     6252
Name: fips, dtype: int64

In [23]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52442
Condominium                                   19342
Duplex (2 Units, Any Combination)              2021
Planned Unit Development                       1947
Quadruplex (4 Units, Any Combination)           729
Triplex (3 Units, Any Combination)              539
Cluster Home                                    335
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       59
Residential General                              42
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

Single Family Residential, Mobile Home, Townhouse, Planned Unit Development, Manufactured, Modular, Prefabricated Home, Residential General, Cooperative      

In [7]:
df

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,buildingclassdesc,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc,parcelid
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,None,NaN,None,261.0,Single Family Residential,NaN,None,NaN,None,14297519
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,None,NaN,None,261.0,Single Family Residential,NaN,None,NaN,None,17052889
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,None,NaN,None,261.0,Single Family Residential,NaN,None,NaN,None,14186244
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,None,2.0,Central,261.0,Single Family Residential,NaN,None,NaN,None,12177905
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,None,2.0,Central,266.0,Condominium,NaN,None,NaN,None,10887214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,673515,11000655,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,None,2.0,Central,261.0,Single Family Residential,NaN,None,NaN,None,11000655
77610,2968375,17239384,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,None,NaN,None,261.0,Single Family Residential,NaN,None,NaN,None,17239384
77611,1843709,12773139,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,None,2.0,Central,261.0,Single Family Residential,NaN,None,NaN,None,12773139
77612,1187175,12826780,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,None,2.0,Central,261.0,Single Family Residential,NaN,None,NaN,None,12826780


## A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).

## A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).

## The output:

## The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.

## hint:

## Look up the dropna documentation.

In [8]:
    def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
        """
        Drops columns and rows from a DataFrame based on the percentage of missing values.

        Args:
            df (pandas.DataFrame): The DataFrame to handle missing values.
            prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
            prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).

        Returns:
            pandas.DataFrame: The DataFrame with dropped rows and columns.
        """

        threshold_col = int(round(prop_required_column * len(df.index)))
        df = df.dropna(thresh=threshold_col, axis=1)


        threshold_row = int(round(prop_required_row * len(df.columns)))
        df = df.dropna(thresh=threshold_row)

        return df


## You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [9]:
df = handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5)

In [10]:
df

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,censustractandblock,id,parcelid,logerror,transactiondate,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,parcelid
0,1727539,14297519,3.5,4.0,NaN,3.5,3100.0,3100.0,6059.0,3.0,...,6.059063e+13,0,14297519,0.025595,2017-01-01,NaN,None,261.0,Single Family Residential,14297519
1,1387261,17052889,1.0,2.0,NaN,1.0,1465.0,1465.0,6111.0,1.0,...,6.111001e+13,1,17052889,0.055619,2017-01-01,NaN,None,261.0,Single Family Residential,17052889
2,11677,14186244,2.0,3.0,NaN,2.0,1243.0,1243.0,6059.0,2.0,...,6.059022e+13,2,14186244,0.005383,2017-01-01,NaN,None,261.0,Single Family Residential,14186244
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,6.037300e+13,3,12177905,-0.103410,2017-01-01,2.0,Central,261.0,Single Family Residential,12177905
4,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,6.037124e+13,4,10887214,0.006940,2017-01-01,2.0,Central,266.0,Condominium,10887214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,6.037101e+13,77609,11000655,0.020615,2017-09-20,2.0,Central,261.0,Single Family Residential,11000655
77610,2968375,17239384,2.0,4.0,NaN,2.0,1612.0,1612.0,6111.0,2.0,...,6.111008e+13,77610,17239384,0.013209,2017-09-21,NaN,None,261.0,Single Family Residential,17239384
77611,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,6.037434e+13,77611,12773139,0.037129,2017-09-21,2.0,Central,261.0,Single Family Residential,12773139
77612,1187175,12826780,2.0,3.0,6.0,2.0,1762.0,1762.0,6037.0,2.0,...,6.037503e+13,77612,12826780,0.007204,2017-09-25,2.0,Central,261.0,Single Family Residential,12826780


In [11]:
df.isnull().sum()

id                                  0
parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           27770
calculatedbathnbr                 616
calculatedfinishedsquarefeet      201
finishedsquarefeet12             3656
fips                                0
fullbathcnt                       616
heatingorsystemtypeid           28008
latitude                            0
longitude                           0
lotsizesquarefeet                8258
propertycountylandusecode           0
propertylandusetypeid               0
propertyzoningdesc              27103
rawcensustractandblock              0
regionidcity                     1472
regionidcounty                      0
regionidzip                        50
roomcnt                             0
unitcnt                         26876
yearbuilt                         270
structuretaxvaluedollarcnt        115
taxvaluedollarcnt                   1
assessmentye

In [12]:
def drop_duplicate_columns(df):
    """
    Drops duplicate columns from a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame to drop duplicate columns from.

    Returns:
        pandas.DataFrame: The DataFrame with duplicate columns removed.
    """
    # Transpose the DataFrame to consider columns as rows for duplicate checking
    transposed_df = df.T
    
    # Drop duplicate columns
    deduplicated_df = transposed_df.drop_duplicates()
    
    # Transpose the DataFrame back to the original shape
    df = deduplicated_df.T
    
    return df


In [13]:
df = drop_duplicate_columns(df)

In [14]:
df

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,1727539,14297519,3.5,4.0,NaN,3.5,3100.0,3100.0,6059.0,3.0,...,1023282.0,2016.0,537569.0,11013.72,60590630072012.0,0,0.025595,2017-01-01,None,Single Family Residential
1,1387261,17052889,1.0,2.0,NaN,1.0,1465.0,1465.0,6111.0,1.0,...,464000.0,2016.0,376000.0,5672.48,61110010023006.0,1,0.055619,2017-01-01,None,Single Family Residential
2,11677,14186244,2.0,3.0,NaN,2.0,1243.0,1243.0,6059.0,2.0,...,564778.0,2016.0,479489.0,6488.3,60590218022012.0,2,0.005383,2017-01-01,None,Single Family Residential
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,145143.0,2016.0,36225.0,1777.51,60373001001006.0,3,-0.10341,2017-01-01,Central,Single Family Residential
4,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,119407.0,2016.0,45726.0,1533.89,60371236012000.0,4,0.00694,2017-01-01,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,354621.0,2016.0,283704.0,4478.43,60371014003002.0,77609,0.020615,2017-09-20,Central,Single Family Residential
77610,2968375,17239384,2.0,4.0,NaN,2.0,1612.0,1612.0,6111.0,2.0,...,67205.0,2016.0,16522.0,1107.48,61110084022016.0,77610,0.013209,2017-09-21,None,Single Family Residential
77611,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,49546.0,2016.0,16749.0,876.43,60374338022005.0,77611,0.037129,2017-09-21,Central,Single Family Residential
77612,1187175,12826780,2.0,3.0,6.0,2.0,1762.0,1762.0,6037.0,2.0,...,522000.0,2016.0,382000.0,6317.15,60375034021020.0,77612,0.007204,2017-09-25,Central,Single Family Residential


In [15]:
df.isnull().sum()

id                                  0
parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           27770
calculatedbathnbr                 616
calculatedfinishedsquarefeet      201
finishedsquarefeet12             3656
fips                                0
fullbathcnt                       616
heatingorsystemtypeid           28008
latitude                            0
longitude                           0
lotsizesquarefeet                8258
propertycountylandusecode           0
propertylandusetypeid               0
propertyzoningdesc              27103
rawcensustractandblock              0
regionidcity                     1472
regionidcounty                      0
regionidzip                        50
roomcnt                             0
unitcnt                         26876
yearbuilt                         270
structuretaxvaluedollarcnt        115
taxvaluedollarcnt                   1
assessmentye

In [16]:
df.propertylandusedesc.value_counts()

Single Family Residential                     52442
Condominium                                   19342
Duplex (2 Units, Any Combination)              2021
Planned Unit Development                       1947
Quadruplex (4 Units, Any Combination)           729
Triplex (3 Units, Any Combination)              539
Cluster Home                                    335
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       59
Residential General                              42
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

## Encapsulate your work inside of functions in a wrangle_zillow.py module.

In [17]:
def new_zillow_data():
    """
    Fetches Zillow data from the database using SQL query.

    Returns:
    - df: DataFrame containing Zillow data
    """
   
    conn = get_db_url('zillow')

    query = '''
        SELECT *
    FROM properties_2017 AS p
    JOIN predictions_2017 AS pr ON p.parcelid = pr.parcelid
    LEFT JOIN airconditioningtype AS act ON p.airconditioningtypeid = act.airconditioningtypeid
    LEFT JOIN architecturalstyletype AS ast ON p.architecturalstyletypeid = ast.architecturalstyletypeid
    LEFT JOIN buildingclasstype AS bct ON p.buildingclasstypeid = bct.buildingclasstypeid
    LEFT JOIN heatingorsystemtype AS hot ON p.heatingorsystemtypeid = hot.heatingorsystemtypeid
    LEFT JOIN propertylandusetype AS plt ON p.propertylandusetypeid = plt.propertylandusetypeid
    LEFT JOIN storytype AS st ON p.storytypeid = st.storytypeid
    LEFT JOIN typeconstructiontype AS tct ON p.typeconstructiontypeid = tct.typeconstructiontypeid
    LEFT JOIN unique_properties AS up ON p.parcelid = up.parcelid
    WHERE p.unitcnt = 1;
    
    '''

    
    df = pd.read_sql(query, conn)
    return df




    def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
        """
        Drops columns and rows from a DataFrame based on the percentage of missing values.

        Args:
            df (pandas.DataFrame): The DataFrame to handle missing values.
            prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
            prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).

        Returns:
            pandas.DataFrame: The DataFrame with dropped rows and columns.
        """

        threshold_col = int(round(prop_required_column * len(df.index)))
        df = df.dropna(thresh=threshold_col, axis=1)


        threshold_row = int(round(prop_required_row * len(df.columns)))
        df = df.dropna(thresh=threshold_row)

        return df



In [18]:
def check_duplicates(df):
    """
    Checks for duplicates in a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame to check for duplicates.

    Returns:
        bool: True if duplicates are found, False otherwise.
    """
    duplicates = df.duplicated('parcelid')
    if duplicates.any():
        return True
    else:
        return False

# Example usage
duplicates_exist = check_duplicates(df)

duplicates_exist

True

In [19]:

def get_most_recent_transactions(df):
    
    duplicates_df = df[df.duplicated(subset='parcelid', keep=False)]

    
    sorted_df = duplicates_df.sort_values(by='transactiondate', ascending=False)

    
    most_recent_transactions = sorted_df.drop_duplicates(subset='parcelid', keep='first')

    return most_recent_transactions




In [20]:
most_recent_transactions = get_most_recent_transactions(df)

In [21]:
most_recent_transactions

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
77613,2455685,13083743,2.0,3.0,4.0,2.0,1050.0,1050.0,6037.0,2.0,...,259334.0,2016.0,182216.0,3345.78,60374081401000.0,77613,-0.197755,2018-05-25,Floor/Wall,Single Family Residential
10543,1094342,11957553,1.0,2.0,3.0,1.0,832.0,832.0,6037.0,1.0,...,455000.0,2016.0,364000.0,5479.5,60371853201005.0,10543,-0.116322,2017-09-19,Floor/Wall,Single Family Residential
28596,1406301,11594130,2.0,2.0,9.0,2.0,1835.0,1835.0,6037.0,2.0,...,238840.0,2016.0,101302.0,3280.52,60377013043009.0,28596,-0.103051,2017-09-19,Central,Condominium
46653,709378,12612211,3.0,3.0,3.0,3.0,1740.0,NaN,6037.0,3.0,...,700000.0,2016.0,420000.0,8779.1,60376700012006.0,46653,0.089218,2017-09-18,Floor/Wall,"Duplex (2 Units, Any Combination)"
7091,1535169,12478591,3.0,4.0,8.0,3.0,2941.0,2941.0,6037.0,3.0,...,612000.0,2016.0,400000.0,7204.93,60375544061005.0,7091,0.039378,2017-09-18,Central,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7132,380957,12575721,2.0,3.0,6.0,2.0,1362.0,1362.0,6037.0,2.0,...,49236.0,2016.0,13435.0,1194.75,60375433042000.0,7132,-0.220062,2017-03-17,Central,Single Family Residential
3540,1403445,10753427,2.0,2.0,7.0,2.0,1054.0,1054.0,6037.0,2.0,...,156062.0,2016.0,26100.0,2728.26,60378002041036.0,3540,-0.145781,2017-03-17,Central,Condominium
9593,1330987,11420117,2.0,3.0,4.0,2.0,1194.0,1194.0,6037.0,2.0,...,244746.0,2016.0,164443.0,3283.2,60376024026006.0,9593,0.01924,2017-03-09,Floor/Wall,Single Family Residential
3198,875806,13973642,2.0,3.0,NaN,2.0,1233.0,1233.0,6059.0,2.0,...,264746.0,2016.0,201959.0,3299.98,60590747013021.0,3198,-0.124305,2017-03-01,None,Single Family Residential
